In [2]:
#New file to vectorise the reviews to perform the data science on them

In [18]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math


<b><h2>Handling of the review data and vectorising of the text

In [4]:
full_data = pd.read_csv("../data/review_dataset.csv")
full_data = pd.DataFrame(full_data)
full_data.dropna()
full_data.drop("Unnamed: 0", axis=1)

,Business Name,Street Number,Route,Postal Town,County,Country,Postcode,Overall Rating,Language,Rating,Review Description,Time,Date,processed review,lemmatized review,Keyword Contains,Keyword No Negative
0,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,have to say this is the best salon ive ever be...,1.652418e+09,2022-05-13 05:07:20,say best salon ive ever 4 weeks lifting acryli...,say best salon ive ever 4 week lifting acryli...,False,False
1,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,nian is incredible ive been coming here for a ...,1.661441e+09,2022-08-25 15:22:45,nian incredible ive coming couple months every...,nian incredible ive coming couple month every...,False,False
2,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,i got a new set of gels from tee who was so so...,1.658412e+09,2022-07-21 14:04:52,got new set gels tee lovely chat helped choose...,got new set gel tee lovely chat helped choose...,False,False
3,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,i have been coming to this salon since i moved...,1.660734e+09,2022-08-17 10:57:00,coming salon since moved birmingham took find ...,coming salon since moved birmingham took find...,False,False
4,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,i had my nails done for the first time and i c...,1.659646e+09,2022-08-04 20:41:01,nails done first time say place super cozy gir...,nail done first time say place super cozy gir...,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22374,Super Hand Car Wash,53-55,Llangollen Road,Wrexham,Wrexham Principal Area,United Kingdom,LL14 3RS,5.0,en,5.0,just had ford ranger polished so pleased with ...,1.654364e+09,2022-06-04 17:31:51,ford ranger polished pleased results vehicle l...,ford ranger polished pleased result vehicle l...,False,False
22375,Super Hand Car Wash,53-55,Llangollen Road,Wrexham,Wrexham Principal Area,United Kingdom,LL14 3RS,5.0,en,5.0,the name says it all fantastic service great p...,1.641225e+09,2022-01-03 15:44:22,name says fantastic service great prices top q...,name say fantastic service great price top qu...,False,False
22376,Super Hand Car Wash,53-55,Llangollen Road,Wrexham,Wrexham Principal Area,United Kingdom,LL14 3RS,5.0,en,5.0,a brilliant super hand car wash drove away spa...,1.652984e+09,2022-05-19 18:06:27,brilliant super hand car wash drove away spark...,brilliant super hand car wash drove away spar...,False,False
22377,Super Hand Car Wash,53-55,Llangollen Road,Wrexham,Wrexham Principal Area,United Kingdom,LL14 3RS,5.0,en,5.0,always happy when my black and yellow fiesta c...,1.654781e+09,2022-06-09 13:28:00,always happy black yellow fiesta comes gleamin...,always happy black yellow fiesta come gleamin...,False,False


In [5]:
from sklearn.model_selection import train_test_split


In [6]:
X_train_text, X_test_text = train_test_split(full_data,test_size=0.33)
y = X_train_text["Keyword Contains"].copy()
test_y = X_test_text["Keyword Contains"].copy()
X_train_text.drop("Keyword Contains",axis=1)
X_test_text.drop("Keyword Contains",axis=1)

,Unnamed: 0,Business Name,Street Number,Route,Postal Town,County,Country,Postcode,Overall Rating,Language,Rating,Review Description,Time,Date,processed review,lemmatized review,Keyword No Negative
19204,19204,Reign Real Estate Limited,23-25,Friar Lane,Leicester,Leicester,United Kingdom,LE1 5QQ,4.0,en,5.0,renting with reign was a great experience sapn...,1.598700e+09,2020-08-29 11:19:18,renting reign great experience sapna team resp...,renting reign great experience sapna team res...,True
2634,2634,American Nail,100,Spring Bank,Hull,Kingston upon Hull,United Kingdom,HU3 1QH,4.4,en,5.0,took his time and my nails look perfect decent...,1.527495e+09,2018-05-28 08:03:10,took time nails look perfect decent price goin...,took time nail look perfect decent price goin...,False
19517,19517,Carter Jonas,256,Banbury Road,Oxford,Oxfordshire,United Kingdom,OX2 7DE,4.7,en,5.0,just picked up a rental in oxford imogen was s...,1.660829e+09,2022-08-18 13:25:02,picked rental oxford imogen super helpful ever...,picked rental oxford imogen super helpful eve...,False
9260,9260,Curtis McKinnell Estate Agents,17,Admirals View,Inverness,Highland Council,United Kingdom,IV2 5GW,5.0,en,5.0,peter was incredibly knowledgeable and efficie...,1.660326e+09,2022-08-12 17:42:25,peter incredibly knowledgeable efficient respo...,peter incredibly knowledgeable efficient resp...,False
1165,1165,Minmins Nails,13,Botchergate,Carlisle,Cumbria,United Kingdom,CA1 1QP,4.8,en,5.0,always love coming in never feel rushed to cho...,1.659124e+09,2022-07-29 19:39:35,always love coming never feel rushed choose de...,always love coming never feel rushed choose d...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4323,4323,File 'N' Style,5,Holland Avenue,Peterborough,Peterborough,United Kingdom,PE4 6EY,5.0,en,5.0,sam was very friendly and did a great job with...,1.503554e+09,2017-08-24 05:48:50,sam friendly great job nails thoughtful checke...,sam friendly great job nail thoughtful checke...,False
15186,15186,SKY HAND CAR WASH,1233,Govan Road,Glasgow,Glasgow City,United Kingdom,G51 4PL,4.8,en,5.0,best car wash in glasgow they actually wash lo...,1.660640e+09,2022-08-16 08:57:29,best car wash glasgow actually wash lol favour...,best car wash glasgow actually wash lol favou...,False
15514,15514,Sparkles Hand Car Wash,150,Aldren street,Lancaster,Lancashire,United Kingdom,LA1 2DE,4.4,en,5.0,they are very goodthey dont only do cars etc b...,1.650813e+09,2022-04-24 15:02:02,goodthey dont cars etc motorbikes scooters goo...,goodthey dont car etc motorbike scooter good job,False
4317,4317,Sunny Nails,2,Park Road,Peterborough,Peterborough,United Kingdom,PE1 2TD,4.0,en,5.0,best pedicure ever lovely professional salon w...,1.651213e+09,2022-04-29 06:16:49,best pedicure ever lovely professional salon f...,best pedicure ever lovely professional salon ...,False


In [7]:
count = 0
for i in X_train_text["Keyword Contains"]:
    if i == True:
        count += 1

print((count/len(X_train_text["Review Description"]))*100)

count = 0
for i in X_test_text["Keyword Contains"]:
    if i == True:
        count += 1

print((count/len(X_test_text["Review Description"]))*100)

4.101914226639098
3.5472515569997296


<h3><b>Sklearn.feature_extraction attempt to vectorise the data



In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD

In [9]:
vectoriser = TfidfVectorizer(ngram_range=(1,5))
train_tfidf = vectoriser.fit_transform(X_train_text["lemmatized review"])
test_tfidf = vectoriser.transform(X_test_text["lemmatized review"])

# full_text = vectoriser.fit_transform(full_data['lemmatized review'])
features = vectoriser.get_feature_names_out()

In [10]:
print("Train Shape: {} Rows and {} Columns".format(*train_tfidf.shape))
print("Test Shape: {} Rows and {} Columns".format(*test_tfidf.shape))

Train Shape: 14993 Rows and 1384719 Columns
Test Shape: 7386 Rows and 1384719 Columns


In [11]:
svd = TruncatedSVD()
svd.fit(train_tfidf)

TruncatedSVD()

In [12]:
X = svd.transform(train_tfidf)
test_tfidf = svd.transform(test_tfidf)

In [13]:
model = LogisticRegression()
model.fit(X,y)

LogisticRegression()

In [14]:
submission = model.predict(test_tfidf)
submission_df = pd.Series(submission).rename("Predicted Exploitation")


In [15]:
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline

In [1]:
def get_models():
    models = dict()
    for i in range(1,30):
        steps = [('svd',TruncatedSVD(n_components=i)),('m',LogisticRegression())]
        models[str(i)] = Pipeline(steps=steps)
    return models    
        
def evaluate_models(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=3,random_state=1)
    scores = cross_val_score(model, X, y, scoring = 'accuracy', cv=cv, n_jobs=-1,error_score="raise")
    return scores



In [16]:
models = get_models()

In [21]:
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_models(model, X, y)
    results.append(scores)
    names.append(name)
    #print('>%s %.3f (%.3f)') % (name, (np.mean(scores), np.std(scores)))

ValueError: n_components == 3, must be <= 2.